In [1]:
from src.classifiers.assemblage_distillation import AssemblageDistillation
from src.classifiers.soft_labelers.m3_soft_labeler import M2SoftLabeler
from src.classifiers.soft_labelers.soft_labeling_functions import LabelingFunctionsModel
from src.classifiers.lr import LR

/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b)
  Referenced from: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torch/lib/libc10.dylib''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
languages = ["es", "en", "fr"]

m3_sl = M2SoftLabeler()
lf_sl = {lang: LabelingFunctionsModel(lang) for lang in languages}
logReg = {lang: LR(15, 3, lang) for lang in languages}
assem_dest = {lang: AssemblageDistillation([m3_sl,lf_sl[lang]], logReg[lang]) for lang in languages}

INFO:m3inference.m3inference:Version 1.1.5
INFO:m3inference.m3inference:Running on cpu.
INFO:m3inference.m3inference:Will use text model. Note that as M3 was optimized to work well with both image and text data,                                     it is not recommended to use text only model unless you do not have the profile image.
INFO:m3inference.m3inference:Model text_model exists at /Users/borjasanchez/m3/models/text_model.mdl.
INFO:m3inference.utils:Checking MD5 for model text_model at /Users/borjasanchez/m3/models/text_model.mdl
INFO:m3inference.utils:MD5s match.
INFO:m3inference.m3inference:Loaded pretrained weight at /Users/borjasanchez/m3/models/text_model.mdl
INFO:root:Creating labeling function model for language: es
INFO:root:Computing es dictionary of names
INFO:root:Creating labeling function model for language: en
INFO:root:Computing en dictionary of names
INFO:root:Creating labeling function model for language: fr
INFO:root:Computing fr dictionary of names


In [3]:
from src.utils import get_data

dev_data = {'en': get_data('data/dev/english_verified.csv'),
            'es': get_data('data/dev/spanish_verified.csv'),
            'fr': get_data('data/dev/french_verified.csv')
           }
for lang, dev in dev_data.items():
    dev["lang"] = lang

In [4]:
L = {lang: assem_dest[lang].train_decision_function(dev) for lang, dev in dev_data.items()}

INFO:root:M3 predicting...
INFO:m3inference.dataset:3466 data entries loaded.
Predicting...:   0%|                                                                                                                                  | 0/7 [00:00<?, ?it/s]/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b)
  Referenced from: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torch/lib/libc10.dylib''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` 

/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b)
  Referenced from: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/borjasanchez/miniforge3/envs/MiniAM2/lib/python3.10/site-packages/torch/lib/libc10.dylib''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Predicting...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:33<00:00,  4.76s/it]
INFO

INFO:root:Applying labeling function to dataset...
INFO:root:Abstains: 3209     Labels: 408
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 37.85it/s]
INFO:root:Training decision function...
INFO:root:Training with 578 sample length


In [5]:
ver = '0.0.0'
[ad.decision_function.save_model_to_path('models', version=ver, save_format='h5') for lang, ad in assem_dest.items()]

[None, None, None]